<a href="https://colab.research.google.com/github/ryanlopezzzz/instagram-bot/blob/jorge_m/rnn_quotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [6]:
text = open('/content/gdrive/MyDrive/Text-Gen/quote.txt', 'r').read()
print(f'length of text: {len(text)} characters')

length of text: 4309536 characters


In [7]:
print(text[:250])



You need a little bit of insanity to do great things.

Honor your joy today, whilst it replenishes and strengthens you - in the company of your Soul.

Give your Friends Roses while they are still alive

Even if there are instances in which it can b


In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')
vocab

76 unique characters


['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [9]:
#Before training, you need to convert the strings to a numerical representation.
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding = 'UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary = list(vocab), mask_token = None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[51, 52, 53, 54, 55, 56, 57], [74, 75, 76]]>

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
tf.strings.reduce_join(chars,axis = -1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis = -1)

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text,'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [15]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))






Y
o
u
 
n
e
e
d


In [16]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder = True)
for seq in sequences.take(1):
  print(chars_from_ids(seq))


tf.Tensor(
[b'\n' b'\n' b'Y' b'o' b'u' b' ' b'n' b'e' b'e' b'd' b' ' b'a' b' ' b'l'
 b'i' b't' b't' b'l' b'e' b' ' b'b' b'i' b't' b' ' b'o' b'f' b' ' b'i'
 b'n' b's' b'a' b'n' b'i' b't' b'y' b' ' b't' b'o' b' ' b'd' b'o' b' '
 b'g' b'r' b'e' b'a' b't' b' ' b't' b'h' b'i' b'n' b'g' b's' b'.' b'\n'
 b'\n' b'H' b'o' b'n' b'o' b'r' b' ' b'y' b'o' b'u' b'r' b' ' b'j' b'o'
 b'y' b' ' b't' b'o' b'd' b'a' b'y' b',' b' ' b'w' b'h' b'i' b'l' b's'
 b't' b' ' b'i' b't' b' ' b'r' b'e' b'p' b'l' b'e' b'n' b'i' b's' b'h'
 b'e' b's' b' '], shape=(101,), dtype=string)


In [17]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text


In [18]:
split_input_target(list('tensorflow'))

(['t', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [19]:
dataset = sequences.map(split_input_target)
dataset

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [20]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\n\nYou need a little bit of insanity to do great things.\n\nHonor your joy today, whilst it replenishes'
Target: b'\nYou need a little bit of insanity to do great things.\n\nHonor your joy today, whilst it replenishes '


In [21]:
BATCH_SIZE = 64
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements)
BUFFER_SIZE = 10000

dataset = (dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [22]:
#Length of the vocabulary in chars
vocab_size = len(vocab)
#The embedding dimension
embedding_dim = 256
#Number of RNN Units
rnn_units = 1024

In [23]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x
model = MyModel (
        #be sure the vocabulary size matches the 'StringLookup' layers.
        vocab_size = len(ids_from_chars.get_vocabulary()), embedding_dim = embedding_dim, rnn_units = rnn_units
)


In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 77) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  19712     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  78925     
                                                                 
Total params: 4,036,941
Trainable params: 4,036,941
Non-trainable params: 0
_________________________________________________________________


In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices


array([27, 63, 21, 74, 69, 21, 21,  6, 72, 15,  6, 61, 66, 38, 44, 71, 41,
       41, 47,  6,  5, 21, 27, 52, 74, 70, 72,  4, 50, 45,  0, 21, 70, 52,
        7, 17, 43,  0, 71, 76, 18, 12, 45, 17, 29, 30, 69, 20, 43, 58, 33,
       52,  0,  9, 75, 13, 21, 69,  4, 66, 24, 23,  4, 43,  4, 31, 72, 23,
       46, 52, 54, 27, 48, 65, 60, 70, 31, 16,  2, 71, 52, 57, 55, 30, 66,
       20, 15, 45, 31,  2, 75, 62, 22, 10, 54, 32,  0, 76, 74, 18])

In [27]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'e present is the vivid fringe of memory tinged with anticipation.\n\nConstitutional democracy, you see'

Next Char Predictions:
 b"Cm9xs99'v3'kpNTuQQW'&9Cbxtv$ZU[UNK]9tb(5S[UNK]uz60U5EFs8ShIb[UNK],y19s$p?;$S$Gv;VbdCXojtG4 ubgeFp83UG yl:-dH[UNK]zx6"


In [28]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [29]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 77)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.3441744


In [30]:
tf.exp(mean_loss).numpy()

77.02842

In [31]:
model.compile(optimizer='adam', loss=loss)

In [32]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [33]:
EPOCHS = 10

In [34]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
666/666 [==============================] - 94s 128ms/step - loss: 1.9460
Epoch 2/10
666/666 [==============================] - 87s 128ms/step - loss: 1.3613
Epoch 3/10
666/666 [==============================] - 86s 127ms/step - loss: 1.2652
Epoch 4/10
666/666 [==============================] - 86s 127ms/step - loss: 1.2169
Epoch 5/10
666/666 [==============================] - 86s 126ms/step - loss: 1.1826
Epoch 6/10
666/666 [==============================] - 86s 126ms/step - loss: 1.1536
Epoch 7/10
666/666 [==============================] - 86s 127ms/step - loss: 1.1274
Epoch 8/10
666/666 [==============================] - 86s 127ms/step - loss: 1.1031
Epoch 9/10
666/666 [==============================] - 86s 127ms/step - loss: 1.0800
Epoch 10/10
666/666 [==============================] - 87s 128ms/step - loss: 1.0586


In [35]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states
    

In [36]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [37]:
start = time.time()
states = None
next_char = tf.constant(["Once upon a time"])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Once upon a time enclan my heart.

People dear to always be beautiful.burning a moment when you are not one of them past it.

Blame the Sun.

Too much listen to a thankful girl is to control the world.

Programmers of a matter of cheapes never then.

The great have a professionally rest of everlasting glorious action arraves later what 79 on Design must provide world comfortly & worship to be Jakes

My kids not make offered a wife there,and until He picking deadlines of oneself

People measured smile at your destination. People care about your nacklew tempesses.

Seek me and now, even if nothing happens about our life.

You must build an entive leader and a job the food.'
she chose meetings and pastier.

Fear is the art of control and life.

When two piccess as important as the only two types of ANG had -but desire for success.

My mom andones your ideas of yourself; touching the stars:you were since you find that more things person.

It has a vision that I love stretched choice.

...p